# Yahoo Fantasy Sports API

In [1]:
# Authenticate
from yahoo_oauth import OAuth2
sc = OAuth2(None, None, from_file='../references/oauth2.json')

[2021-01-23 10:36:15,941 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 10:36:15,952 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 4.649432897567749
[2021-01-23 10:36:15,953 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [2]:
import yahoo_fantasy_api as yfa
gm = yfa.Game(sc, 'nba')

In [2]:
# Get League Ids I have access to
league_id_list = gm.league_ids(year=2020)
league_id = "".join(str(id) for id in league_id_list)
print(f"The League Id is: {league_id}")

NameError: name 'gm' is not defined

In [107]:
# Categories
lg = gm.to_league(league_id)
league_categories = lg.stat_categories()

In [5]:
# Print league categories
print(f'The 9 categories in the league are: '
      f'{[category["display_name"] for category in league_categories]}')

The 9 categories in the league are: ['FG%', 'FT%', '3PTM', 'PTS', 'REB', 'AST', 'ST', 'BLK', 'TO']


In [6]:
tms = lg.teams()
tms

{'402.l.55374.t.3': {'team_key': '402.l.55374.t.3',
  'team_id': '3',
  'name': 'CrabbeHerByThePussy',
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/3',
  'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://s.yimg.com/cv/apiv2/default/nba/nba_1_m.png'}}],
  'waiver_priority': 8,
  'number_of_moves': '4',
  'number_of_trades': 0,
  'roster_adds': {'coverage_type': 'week', 'coverage_value': 5, 'value': '0'},
  'league_scoring_type': 'headone',
  'draft_position': 5,
  'has_draft_grade': 0,
  'managers': [{'manager': {'manager_id': '3',
     'nickname': 'Marcos',
     'guid': 'MQNQZBBOR5XNH4WSVWGNPV4CCU',
     'email': 'marcostapiachile@gmail.com',
     'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg'}}]},
 '402.l.55374.t.7': {'team_key': '402.l.55374.t.7',
  'team_id': '7',
  'name': 'Russty Cuntbrooks',
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/7',
  'team_logos': [{'team_logo': {'size': 'large',
     'ur

In [7]:
tm = lg.to_team('402.l.55374.t.6')
tm

In [15]:
tm.roster(1)

[{'player_id': 5650,
  'name': 'Malik Beasley',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['PG', 'SG', 'G', 'Util'],
  'selected_position': 'PG'},
 {'player_id': 6021,
  'name': 'Mikal Bridges',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'],
  'selected_position': 'SG'},
 {'player_id': 5764,
  'name': 'Lonzo Ball',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['PG', 'SG', 'G', 'Util'],
  'selected_position': 'G'},
 {'player_id': 6174,
  'name': 'PJ Washington',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['SF', 'PF', 'F', 'Util'],
  'selected_position': 'SF'},
 {'player_id': 5318,
  'name': 'Julius Randle',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['PF', 'F', 'C', 'Util'],
  'selected_position': 'PF'},
 {'player_id': 3704,
  'name': 'LeBron James',
  'status': '',
  'position_type': 'P',
  'eligible_positions': ['PG', 'SG', 'G', 'SF', 'F', 'Util'],
  'sel

In [3]:
tm = lg.to_team('402.l.55374.t.4')
tm.roster(1)

NameError: name 'lg' is not defined

In [10]:
tms = lg.teams()
tms


{'402.l.55374.t.3': {'team_key': '402.l.55374.t.3',
  'team_id': '3',
  'name': 'CrabbeHerByThePussy',
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/3',
  'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://s.yimg.com/cv/apiv2/default/nba/nba_1_m.png'}}],
  'waiver_priority': 8,
  'number_of_moves': '4',
  'number_of_trades': 0,
  'roster_adds': {'coverage_type': 'week', 'coverage_value': 5, 'value': '0'},
  'league_scoring_type': 'headone',
  'draft_position': 5,
  'has_draft_grade': 0,
  'managers': [{'manager': {'manager_id': '3',
     'nickname': 'Marcos',
     'guid': 'MQNQZBBOR5XNH4WSVWGNPV4CCU',
     'email': 'marcostapiachile@gmail.com',
     'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg'}}]},
 '402.l.55374.t.7': {'team_key': '402.l.55374.t.7',
  'team_id': '7',
  'name': 'Russty Cuntbrooks',
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/7',
  'team_logos': [{'team_logo': {'size': 'large',
     'ur

In [12]:
for key, vav in tms.items():
    a = vav["name"]
    print(a)

CrabbeHerByThePussy
Russty Cuntbrooks
McCurry
Nunn of Y'all Betta
Magic's Johnson
El Ladrón de Cabras
Autopick
Wakanda Forever
Swaggy P
yOBItch Toppin Me
LaLaLand
Tvon's Tip-Top Team


In [ ]:
lg.free_agents()



In [13]:
GUARD = "G"
FORWARD = "F"
CENTER = "C"

POSITIONS_LIST = [GUARD, FORWARD, CENTER]
fa_CF = lg.free_agents('G')

In [14]:
fa_CF


[{'player_id': 3826,
  'name': 'Andre Iguodala',
  'position_type': 'P',
  'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'],
  'percent_owned': 1,
  'status': ''},
 {'player_id': 3860,
  'name': 'Trevor Ariza',
  'status': 'INJ',
  'position_type': 'P',
  'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'],
  'percent_owned': 3},
 {'player_id': 4139,
  'name': 'JJ Redick',
  'position_type': 'P',
  'eligible_positions': ['SG', 'G', 'Util'],
  'percent_owned': 14,
  'status': ''},
 {'player_id': 4149,
  'name': 'Rajon Rondo',
  'position_type': 'P',
  'eligible_positions': ['PG', 'G', 'Util'],
  'percent_owned': 3,
  'status': ''},
 {'player_id': 4471,
  'name': 'D.J. Augustin',
  'position_type': 'P',
  'eligible_positions': ['PG', 'G', 'Util'],
  'percent_owned': 1,
  'status': ''},
 {'player_id': 4633,
  'name': 'Wayne Ellington',
  'position_type': 'P',
  'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'],
  'percent_owned': 5,
  'status': ''},
 {'player_id': 4651,
  'name': 

## Team Rosters
This section will create dataframes for each team in the league.

In [38]:
tms = lg.teams()

for team_id in tms:
    tm = lg.to_team(team_id)
    print(tm.roster())

[{'player_id': 3930, 'name': 'Chris Paul', 'status': '', 'position_type': 'P', 'eligible_positions': ['PG', 'G', 'Util'], 'selected_position': 'PG'}, {'player_id': 6048, 'name': 'Gary Trent Jr.', 'status': '', 'position_type': 'P', 'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'], 'selected_position': 'SG'}, {'player_id': 6401, 'name': 'Deni Avdija', 'status': 'INJ', 'position_type': 'P', 'eligible_positions': ['SG', 'G', 'SF', 'F', 'Util'], 'selected_position': 'G'}, {'player_id': 4468, 'name': 'Danilo Gallinari', 'status': 'INJ', 'position_type': 'P', 'eligible_positions': ['SF', 'PF', 'F', 'Util'], 'selected_position': 'SF'}, {'player_id': 5154, 'name': 'Otto Porter Jr.', 'status': '', 'position_type': 'P', 'eligible_positions': ['SF', 'PF', 'F', 'Util'], 'selected_position': 'PF'}, {'player_id': 5487, 'name': 'Larry Nance Jr.', 'status': '', 'position_type': 'P', 'eligible_positions': ['PF', 'F', 'C', 'Util'], 'selected_position': 'F'}, {'player_id': 5497, 'name': 'Richaun Holm

In [135]:
tm = lg.to_team('402.l.55374.t.5')
team_roster = tm.roster()
player_in_team_list = [player["name"] for player in team_roster]
player_in_team_list

['Ben Simmons',
 'Justin Holiday',
 'DeMar DeRozan',
 'Keldon Johnson',
 'Willie Cauley-Stein',
 'Jerami Grant',
 'Robert Williams',
 'Mitchell Robinson',
 'Andre Drummond',
 'DeMarcus Cousins',
 'Giannis Antetokounmpo',
 'Ricky Rubio',
 'Rui Hachimura',
 'Hamidou Diallo',
 'Caris LeVert']

In [140]:
from src.data import player_9cat_average as p9ca

p9ca.get_player_9cat_season_average("Robert Williams")

IndexError: list index out of range

In [111]:
import pandas as pd
test = pd.DataFrame()
for player in player_in_team_list:
    test = test.append(p9ca.get_player_9cat_season_average(player))

In [112]:
test.reset_index(inplace=True, drop=True)

In [113]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PLAYER  15 non-null     object
 1   FG_PCT  15 non-null     object
 2   FT_PCT  15 non-null     object
 3   FG3M    15 non-null     object
 4   PTS     15 non-null     object
 5   REB     15 non-null     object
 6   AST     15 non-null     object
 7   STL     15 non-null     object
 8   BLK     15 non-null     object
 9   TOV     15 non-null     object
 10  GAMES   15 non-null     int64 
dtypes: int64(1), object(10)
memory usage: 1.4+ KB


In [114]:
test.columns[1:]

for column in test.columns[1:]:
    test[column] = test[column].astype(float)

In [115]:
test.loc["mean"] = test.mean()

In [116]:
test


,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,GAMES
0,Patty Mills,0.4630,0.4830,2.900000,14.100000,1.60,3.10,0.300000,0.10,0.800000,15.000000
1,Marcus Smart,0.3440,0.5700,1.800000,13.200000,2.80,5.50,1.500000,0.50,1.600000,12.000000
2,Malcolm Brogdon,0.4670,0.8090,3.000000,22.500000,3.90,7.30,1.700000,0.10,1.700000,14.000000
3,Will Barton,0.4080,0.2860,1.600000,11.400000,4.20,3.10,1.100000,0.60,1.100000,14.000000
4,Nikola Jokic,0.5910,0.8270,1.200000,25.100000,11.40,10.00,1.900000,0.60,4.000000,14.000000
5,Duncan Robinson,0.4570,0.6000,3.800000,14.800000,3.80,1.60,0.500000,0.40,1.500000,13.000000
6,Pascal Siakam,0.4420,0.7740,1.200000,18.800000,7.70,4.70,0.800000,0.60,2.500000,13.000000
7,Wendell Carter Jr.,0.5260,0.7070,0.300000,12.800000,8.00,2.40,0.600000,0.60,1.900000,14.000000
8,Jeremy Lamb,0.5000,1.0000,1.000000,10.000000,2.00,0.00,2.000000,0.00,1.000000,1.000000
9,Cameron Johnson,0.4780,0.3080,2.500000,11.800000,3.20,1.70,0.700000,0.50,0.800000,13.000000


In [127]:
lg.teams()["402.l.55374.t.12"]

{'team_key': '402.l.55374.t.12',
 'team_id': '12',
 'name': 'El Ladrón de Cabras',
 'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/12',
 'team_logos': [{'team_logo': {'size': 'large',
    'url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/007f22590f40253091e62423f37483223bbc95b2f92b061318f86a455de84ef0.jpg'}}],
 'waiver_priority': 12,
 'number_of_moves': '2',
 'number_of_trades': 0,
 'roster_adds': {'coverage_type': 'week', 'coverage_value': 5, 'value': '0'},
 'league_scoring_type': 'headone',
 'draft_position': 3,
 'has_draft_grade': 0,
 'managers': [{'manager': {'manager_id': '12',
    'nickname': 'Blake',
    'guid': 'P6ND4NTO4LWPTQEYUA5SYLPW3U',
    'email': 'blakewhitelacourse@yahoo.ca',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg'}}]}

In [57]:
import time
import random
import pandas as pd
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
from src.data import player_9cat_average as p9ca

NBA = "nba"
SEASON = 2020
JSON_FOLDER = "../references/oauth2.json"
# Team dictionaries
LALALAND = {'team_key': '402.l.55374.t.1', 'name': 'LaLaLand'}
AUTOPICK = {'team_key': '402.l.55374.t.2', 'name': 'Autopick'}
CRABBEHERBYTHEPUSSY = {'team_key': '402.l.55374.t.3', 'name': 'CrabbeHerByThePussy'}
MAGICS_JOHNSON = {'team_key': '402.l.55374.t.4', 'name': "Magic's Johnson"}
MCCURRY = {'team_key': '402.l.55374.t.5', 'name': 'McCurry'}
NUNN_OF_YALL_BETTA = {'team_key': '402.l.55374.t.6', 'name': "Nunn of Y'all Betta"}
RUSTY_CUNTBROOKS = {'team_key': '402.l.55374.t.7', 'name': 'Russty Cuntbrooks'}
WAKANDA_FOREVER = {'team_key': '402.l.55374.t.8', 'name': 'Wakanda Forever'}
SWAGGY_P = {'team_key': '402.l.55374.t.9', 'name': 'Swaggy P'}
YOBITCH_TOPPIN_ME = {'team_key': '402.l.55374.t.10', 'name': 'yOBItch Toppin Me'}
TVONS_TIP_TOP_TEAM = {'team_key': '402.l.55374.t.11', 'name': "Tvon's Tip-Top Team"}
EL_LADRON_DE_CABRAS = {'team_key': '402.l.55374.t.12', 'name': 'El Ladrón de Cabras'}
league_team_list = [LALALAND, AUTOPICK, CRABBEHERBYTHEPUSSY, MAGICS_JOHNSON, MCCURRY,
                    NUNN_OF_YALL_BETTA, RUSTY_CUNTBROOKS, WAKANDA_FOREVER, SWAGGY_P,
                    YOBITCH_TOPPIN_ME, TVONS_TIP_TOP_TEAM, EL_LADRON_DE_CABRAS]

NAME_KEY = "name"
GAMES_COLUMN = "GAMES"
MEAN_ROW = "mean"
TEAM_KEY = "team_key"
PLAYER_COLUMN = "PLAYER"
TEAM_COLUMN = "TEAM"

def yahoo_fantasy_api_authentication():

    sc = OAuth2(None, None, from_file=JSON_FOLDER)

    return sc

def yahoo_fantasy_league(sc):

    gm = yfa.Game(sc, NBA)
    league_id_list = gm.league_ids(year=SEASON)
    league_id = "".join(str(id) for id in league_id_list)
    league = gm.to_league(league_id)

    return league


def create_roster_dataframe(player_list):

    active_players = p9ca.nba_active_players()
    roster_dataframe = pd.DataFrame()
    for player in player_list:
        try:
            roster_dataframe = roster_dataframe.append(p9ca.player_average_9cat_stats(player,
                                                                                      active_players))
        except:
            print(f"Could not fetch stats for player {player}")
    roster_dataframe.reset_index(inplace=True, drop=True)

    return roster_dataframe

def format_roster_dataframe(roster_dataframe):

    for column in roster_dataframe.columns[1:]:
        roster_dataframe[column] = roster_dataframe[column].astype(float)

    roster_dataframe.loc[MEAN_ROW] = roster_dataframe.mean()
    roster_dataframe = p9ca.format_dataframe_decimals(roster_dataframe)
    roster_dataframe.drop(GAMES_COLUMN, axis=1, inplace=True)

    return roster_dataframe

def team_9cat_average_stats(team_dictionary, league):

    team = league.to_team(team_dictionary[TEAM_KEY])
    team_roster = team.roster()
    player_in_team_list = [player[NAME_KEY] for player in team_roster]

    roster_dataframe = create_roster_dataframe(player_in_team_list)

    roster_dataframe = format_roster_dataframe(roster_dataframe)

    return roster_dataframe


def fantasy_team_mean_stats(roster_dataframe, team_dictionary):

    mean_series = roster_dataframe.loc[MEAN_ROW]
    mean_series[PLAYER_COLUMN] = team_dictionary[NAME_KEY]
    team_mean_dataframe = pd.DataFrame(mean_series).T
    team_mean_dataframe = team_mean_dataframe.rename(columns={PLAYER_COLUMN : TEAM_COLUMN})
    team_mean_dataframe.reset_index(inplace=True, drop=True)

    return team_mean_dataframe


def player_to_team_mean_stats(team_dictionary, league):
    """
    This function creates a dataframe containing the season averaged stats for each player on a
    Fantasy Team. It then finds the average stats for the entire team and returns a single row
    dataframe with the Fantasy Team name and its seasoned averaged 9cat stats.
    """

    roster_dataframe = team_9cat_average_stats(team_dictionary, league)
    team_mean_dataframe = fantasy_team_mean_stats(roster_dataframe, team_dictionary)

    return team_mean_dataframe


def league_averages(league_team_list):

    league_averages_dataframe = pd.DataFrame()
    for team in league_team_list:
        yahoo_fantasy_api_authentication()
        league_averages_dataframe = league_averages_dataframe.append(player_to_team_mean_stats
                                                                     (team, league))
    league_averages_dataframe.reset_index(drop=True, inplace=True)
    return league_averages_dataframe

In [32]:
# Single team average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
team_9cat_average_stats(CRABBEHERBYTHEPUSSY, league)


[2021-01-23 10:16:05,522 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 10:16:05,524 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2844.6850221157074
[2021-01-23 10:16:05,526 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


Could not fetch stats for player Deni Avdija


,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Chris Paul,0.440,0.700,0.9,13.3,4.4,8.7,0.9,0.3,2.3
1,Norman Powell,0.386,0.470,2.0,12.5,2.1,0.7,1.0,0.2,1.3
2,Otto Porter Jr.,0.481,0.545,1.8,12.5,6.8,1.9,0.7,0.1,1.1
3,John Collins,0.520,0.554,1.0,15.9,7.5,1.5,0.6,1.0,1.3
4,Tobias Harris,0.526,0.731,2.1,19.6,6.8,3.1,1.1,0.9,2.2
5,Bam Adebayo,0.607,0.858,0.0,18.6,9.2,5.2,0.8,1.0,3.5
6,Karl-Anthony Towns,0.479,0.958,1.5,22.0,12.5,4.2,0.5,2.8,2.5
7,Larry Nance Jr.,0.505,0.378,1.6,11.1,6.5,3.7,2.4,0.5,1.7
8,Nicolas Batum,0.498,0.479,2.1,10.2,4.9,2.5,1.3,0.4,0.7
9,Myles Turner,0.479,0.587,1.4,13.2,7.0,1.1,1.4,4.1,1.4


In [2]:
%%time
# League average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
league_averages_dataframe = league_averages(league_team_list)
league_averages_dataframe

[2021-01-23 11:41:19,397 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 11:41:19,402 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3908.0990290641785
[2021-01-23 11:41:19,403 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-01-23 11:41:19,404 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2021-01-23 11:41:20,371 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 11:41:20,382 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 0.9768552780151367
[2021-01-23 11:41:20,383 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-01-23 11:42:11,753 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 11:42:11,757 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 52.352134466171265
[2021-01-23 11:42:11,758 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-01-23 11:43:15,503 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 11:43:15,507 DEBUG] [yahoo_oauth.oauth.token_is_va

Wall time: 0 ns
Could not fetch stats for player James Wiseman
Could not fetch stats for player LaMelo Ball
Could not fetch stats for player Deni Avdija
Could not fetch stats for player Tyrese Haliburton
Could not fetch stats for player PJ Washington
Could not fetch stats for player Robert Williams
Could not fetch stats for player Cole Anthony
Could not fetch stats for player Obi Toppin
Could not fetch stats for player Anthony Edwards
Could not fetch stats for player Precious Achiuwa
Could not fetch stats for player Isaac Okoro


## Yahoo Player Info
This section will create a function which will retrieve an NBA players team and jersey number.
These identifiers will then be used to find the player in the Stats NBA API. This is done because
 the current method of searching for a players stats on the Stats NBA API using the Yahoo Fantasy
  name can lead to errors. For example, on Yahoo it is PJ Washington, on Stats NBA it is P.J.
  Washington.

In [58]:
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)

[2021-01-23 20:30:45,482 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 20:30:45,484 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3807.6330931186676
[2021-01-23 20:30:45,485 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-01-23 20:30:45,486 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [54]:
EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"

def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number

In [46]:
players_team, players_number = yahoo_player_team_and_jersey("Robert Williams")
print(players_team, players_team)

In [47]:
from nba_api.stats.endpoints import commonallplayers

COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"

def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == players_team]

    return player_ids_in_team_list

In [48]:
player_ids_in_team_list = player_ids_by_nba_team_name(players_team)
player_ids_in_team_list

In [49]:
from nba_api.stats.endpoints import commonplayerinfo

COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"

def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name

In [50]:
nba_player_name_from_jersey_search(player_ids_in_team_list, players_number)

'Robert Williams III'

In [61]:
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo

EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
JERSEY_KEY = "uniform_number"
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"


def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number


def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == players_team]

    return player_ids_in_team_list


def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name


def yahoo_name_to_nba_name(yahoo_player_name):
    """ Function expectes a Yahoo player name and it will find the players Team Name and Jersey
    Number. It then searches the NBA API for Player IDs belonging to found Team Name. For every
    Player ID it searches their Jersey Number. When there is a match, it returns the NBA Player
    Name.
    """

    player_team, player_number = yahoo_player_team_and_jersey(yahoo_player_name)
    player_ids_in_team_list = player_ids_by_nba_team_name(player_team)
    nba_player_name = nba_player_name_from_jersey_search(player_ids_in_team_list, player_number)

    return nba_player_name

In [62]:
yahoo_name_to_nba_name("Robert Williams")


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)